In [1]:
!pip install google-api-python-client

  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached proto_plus-1.25.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB 487.6 kB/s eta 0:00:27
   ---------------------------------------- 0.1/12.8 MB 751.6 kB/s eta 0:00:17
   ---------------------------------------- 0.1/12.8 MB 798.9 kB/s eta 0:00:16
    --------------------------------------- 0.2/12.8 MB 807.1 kB/s eta 0:00:16
    --------------------------------------- 0.2/12.8 MB 860.2 kB/s eta 0:00:15
    --------------------------------------- 0.3/12.8 MB 911.0 kB/s eta 0:00:14
   - ------------------------------------

In [35]:
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [49]:
cred=pd.read_csv('ytb_key.csv')

In [50]:
cred=cred['API_KEY'].values[0]

In [51]:
API_KEY=cred

In [52]:
def get_trending_videos(api_key, max_results=200):
    try:
        # Build the YouTube service
        youtube = build('youtube', 'v3', developerKey=api_key)

        # Initialize the list to hold video details
        videos = []

        # Fetch the most popular videos
        request = youtube.videos().list(
            part='snippet,contentDetails,statistics',
            chart='mostPopular',
            regionCode='IN',  
            maxResults=50
        )

        # Paginate through the results if max_results > 50
        while request and len(videos) < max_results:
            response = request.execute()
            for item in response['items']:
                video_details = {
                    'video_id': item['id'],
                    'title': item['snippet']['title'],
                    'description': item['snippet']['description'],
                    'published_at': item['snippet']['publishedAt'],
                    'channel_id': item['snippet']['channelId'],
                    'channel_title': item['snippet']['channelTitle'],
                    'category_id': item['snippet']['categoryId'],
                    'tags': item['snippet'].get('tags', []),
                    'duration': item['contentDetails']['duration'],
                    'definition': item['contentDetails']['definition'],
                    'caption': item['contentDetails'].get('caption', 'false'),
                    'view_count': item['statistics'].get('viewCount', 0),
                    'like_count': item['statistics'].get('likeCount', 0),
                    'dislike_count': item['statistics'].get('dislikeCount', 0),
                    'favorite_count': item['statistics'].get('favoriteCount', 0),
                    'comment_count': item['statistics'].get('commentCount', 0)
                }
                videos.append(video_details)

            # Get the next page token
            request = youtube.videos().list_next(request, response)

        return videos[:max_results]
    
    except HttpError as e:
        print(f'An HTTP error {e.resp.status} occurred: {e.content}')
        return []

In [53]:
def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    return df

In [56]:
def main():
    trending_videos = get_trending_videos(API_KEY)
    if trending_videos:  # Check if any data was returned
        filename = 'trending_videos.csv'
        df=save_to_csv(trending_videos, filename)
        print(df)
        print(f'Trending videos saved to {filename}')
    else:
        print('No data to save.')

In [57]:
if __name__ == '__main__':
    main()

        video_id                                              title  \
0    WR8PyAhn6tQ  Vidaamuyarchi - Pathikichu Lyric | Ajith Kumar...   
1    gzIzlhxBRU4  Try Not to Laugh or Dare Challenge vs Ruchika ...   
2    AATu-TO7trE  Yezhu Kadal Yezhu Malai - Trailer | Ram | Nivi...   
3    AUyyB5peYck                     I MADE SQUID GAME IN MINECRAFT   
4    VLGo6QSFtnU                           BIGG BOSS 18 WAS A SCAM?   
..           ...                                                ...   
111  7xALolZzhSM  Marvel Television's Daredevil: Born Again | Of...   
112  0IjWsnAyQLg  Sunn Mere Dil EP 31 [Eng Sub] Digitally Presen...   
113  qJGfydJ2q1M                       MY FIRST DAY IN STORAGE WARS   
114  6caGIi2JSds  Manisha Rani ko propose kiya Gauarav Khanna ne...   
115  swr9lJQwe2w  Granny In House || Escape Together || Tejasvi ...   

                                           description          published_at  \
0    Lyca Productions \nSubaskaran Presents\nAjith ...  2025-01-19T